In [2]:
from __future__ import print_function
import keras
from scipy.io import loadmat
import tensorflow as tf
from numpy.random import seed as np_seed, choice
import csv
import tensorflow as tf
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
#from keras.wrappers.scikit_learn import KerasClassifier
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from keras import utils
from sklearn.model_selection import cross_val_score,train_test_split,KFold
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.optimizers import SGD
from keras.layers import Dense,LSTM, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D,BatchNormalization
from keras.models import load_model
from sklearn import preprocessing
#from tensorflow.keras.models import Model, Input
from tensorflow.keras import Input, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Flatten,Permute,concatenate
from keras.layers import Conv2D, MaxPooling2D,Conv1D
from keras.layers import Concatenate, Reshape,add
import os
from fb_read   import prepare_data_as, prepare_template, normalize
import math
from math import pi
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from keras.layers import Layer
from functools import reduce
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2024-05-18 07:12:41.025619: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-18 07:12:41.203554: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 07:12:41.203579: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 07:12:41.204430: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-18 07:12:41.282897: I tensorflow/core/platform/cpu_feature_g

In [3]:
class Coff2(Layer):

    def __init__(self, params, **kwargs):
        self.tw = params['tw']
        self.Fs = params['Fs']
        self.cl = params['cl']
        self.corr = None
        super(Coff2, self).__init__(**kwargs)

    def build(self, input_shape):
        # No trainable weight variable for this layer.
        super(Coff2, self).build(input_shape)

    def call(self, input, **kwargs):
        x = input[0] # [?,tw] signal
        t = input[1] # [?,tw,cl] reference
        self.corr = K.sign(x) * x ** 2 + K.sign(t) * t ** 2
        self.out = self.corr
        #self.out = K.expand_dims(self.out, -1)
        return self.out

    # def compute_output_shape(self, input_shape):
    #     return K.int_shape(self.out)

In [4]:
class Self_Attention(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        print('output_dim' + str(output_dim))  # 40
        super(Self_Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # 为该层创建一个可训练的权重
        # inputs.shape = (batch_size, time_steps, seq_len)
        print('input_shape'+str(input_shape))  # input_shape[(None, 40, 50), (None, 40, 50)]
        self.kernel = self.add_weight(name='kernel',
                                      shape=(3, input_shape[0][2], 1),
                                      initializer='uniform',
                                      trainable=True)
        print('self.kernel.shape:' + str(self.kernel.shape))  # self.kernel.shape:(3, 50, 40)
        super(Self_Attention, self).build(input_shape)  # 一定要在最后调用它

    def call(self, x):
        WQ = K.dot(x[0], self.kernel[0])
        WK = K.dot(x[0], self.kernel[1])
        WV = K.dot(x[1], self.kernel[2])

        print("WK.shape", WK.shape)  # WQ.shape (None, 40, 1)

        print("K.permute_dimensions(WK, [0, 2, 1]).shape", K.permute_dimensions(WK, [0, 2, 1]).shape)
        # K.permute_dimensions(WK, [0, 2, 1]).shape(None, 40, 40)
        QK = K.batch_dot(WQ, K.permute_dimensions(WK, [0, 2, 1]))  # (40, 40)
        QK = QK / (50 ** 0.5)
        QK = K.softmax(QK)
        print("QK.shape", QK.shape)  # QK.shape (None, 40, 40)
        V = K.batch_dot(QK, WV)
        print('V.shape:' + str(V.shape)) # V.shape:(None, 40, 1)
        self.V = Reshape((40,))(V)

        return self.V

    # def compute_output_shape(self, input_shape):
    #     return K.int_shape(self.V)
## correlation analysis layer

In [5]:
class Coff(Layer):

    def __init__(self, params, **kwargs):
        self.tw = params['tw']
        self.Fs = params['Fs']
        self.cl = params['cl']
        self.corr = None
        #self.out=self.corr
        super(Coff, self).__init__(**kwargs)

    def build(self, input_shape):
        # No trainable weight variable for this layer.
        super(Coff, self).build(input_shape)

    def call(self, input, **kwargs):
        x = input[0] # [?,tw] signal
        t = input[1] # [?,tw,cl] reference
        t_ = K.reshape(t,(-1,self.tw,self.cl))
        print(t_.shape)

        corr_xt = K.batch_dot(x,t_,axes=(1,1)) # [?,cl]
        corr_xx = K.batch_dot(x,x,axes=(1,1)) # [?,1]
        corr_tt = K.sum(t_*t_,axis=1) # [?,cl]
        self.corr = corr_xt/K.sqrt(corr_tt)/K.sqrt(corr_xx)
        self.out = self.corr
        #self.out = K.expand_dims(self.out, -1)
        return self.out

    def compute_output_shape(self, input_shape):
        return K.int_shape(self.out)

In [6]:
def write_log(filepath='test.log', data=[], mode='w'):
        '''
        filepath: path to save
        data: list of data
        mode: a = update data to file, w = write a new file
        '''
        try:
            with open(filepath, mode) as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(data)
        except IOError:
            raise Exception('I/O error')

In [8]:
## parameters
# channels: Pz, PO5, PO3, POz, PO4, PO6, O1, Oz, O2
permutation = [47,53,54,55,56,57,60,61,62]
params = {'tw':250,'Fs':250,'cl':40,'ch':len(permutation)}
# tf.random.set_seed(1122)
# np_seed(1122)
root_dir='./Bench/'
all_freqs = loadmat(root_dir+'Freq_Phase.mat')['freqs'][0] # true fre

In [9]:
params['tw']

250

In [ ]:
for subj in range(1,36):
    print('subject id is: '+str(subj))
    #train_run = [1,2,3,4,0]
    #test_run = [5]
    run=[0,1,2,3,4,5]
    for i in range(len(run)):
        test_run=[run[i]]
        train_run=run[:i]+run[i+1:]
        print(f'epoch第{i+1}次')
        # for k, (train_run, test_run) in enumerate(kfold):
        #print(test_run)
        file_list = [root_dir+"/x_train1_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/x_train2_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/x_train3_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/y_train_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/x_test1_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/x_test2_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/x_test3_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/y_test_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/template1_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/template2_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     root_dir+"/template3_file" + str(subj) + "_" + str(params['tw']) + "_" + str(test_run) + ".npy",
                     ]
        x_train1_file, x_train2_file,x_train3_file,yy_train_file, x_test1_file,x_test2_file,x_test3_file, yy_test_file, template1_file,template2_file,template3_file = file_list
        if not reduce(lambda x, y: os.path.exists(x) and os.path.exists(y), file_list):
            x_train1,x_train2,x_train3 ,y_train,freq = prepare_data_as(subj,train_run,params['tw']) ## [?,tw,ch]
            x_test1,x_test2,x_test3,y_test,__ = prepare_data_as(subj,test_run,params['tw']) # [?,tw,ch]
    
            x_train1 = x_train1.reshape((x_train1.shape[0],params['tw'],params['ch'],1))
            x_test1 = x_test1.reshape((x_test1.shape[0],params['tw'],params['ch'],1))
            x_train2 = x_train2.reshape((x_train2.shape[0], params['tw'], params['ch'], 1))
            x_test2 = x_test2.reshape((x_test2.shape[0], params['tw'], params['ch'], 1))
            x_train3 = x_train3.reshape((x_train3.shape[0], params['tw'], params['ch'], 1))
            x_test3 = x_test3.reshape((x_test3.shape[0], params['tw'], params['ch'], 1))
            y_train = keras.utils.to_categorical(y_train, params['cl'])
            y_test = keras.utils.to_categorical(y_test, params['cl'])
            print("y_test=",y_test)
            ## build reference signal
            template, template2, template3 = prepare_template(subj, train_run, params['tw'])  # [cl*sample,cl,tw,ch]
            template = np.transpose(template, axes=(0, 2, 1, 3))  # [cl*sample,tw,cl,ch]
            template2 = np.transpose(template2, axes=(0, 2, 1, 3))  # [cl*sample,tw,cl,ch]
            template3 = np.transpose(template3, axes=(0, 2, 1, 3))  # [cl*sample,tw,cl,ch]
            data_list = [x_train1,x_train2,x_train3, y_train, x_test1,x_test2,x_test3, y_test, template,template2,template3]
            for i, data in enumerate(data_list):
                np.save(file_list[i], data)
        else:
                x_train1,x_train2,x_train3, y_train, x_test1,x_test2,x_test3, y_test, template,template2,template3 = [np.load(file) for file in file_list]
    
    
        freq = np.array(all_freqs)
        # Y = K.zeros(10, params['tw'], params['ch']);
        Y = np.zeros([10, params['tw'], 40], np.float64)
        pha_val = np.array([0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5
                               , 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5, 0, 0.5, 1, 1.5]) * math.pi
        sig_len = params['tw'] * params['Fs']
        t = [i / 1000 for i in range(4, 1001, 4)]
        t = np.array(t)
        for targ_i in range(0, 40):
            # for Nhar in range(1, 2):
            # print( 2 * math.pi * freq[targ_i] * 1 * pha_val[targ_i] )
            Y0 = np.array([np.sin(2 * math.pi * 1 * freq[targ_i] * t + 1 * pha_val[targ_i]),
                               np.cos(2 * math.pi * 1 * freq[targ_i] * t + 1 * pha_val[targ_i]),
                               np.sin(2 * math.pi * 2 * freq[targ_i] * t + 2 * pha_val[targ_i]),
                               np.cos(2 * math.pi * 2 * freq[targ_i] * t + 2 * pha_val[targ_i]),
                               np.sin(2 * math.pi * 3 * freq[targ_i] * t + 3 * pha_val[targ_i]),
                               np.cos(2 * math.pi * 3 * freq[targ_i] * t + 3 * pha_val[targ_i]),
                               np.sin(2 * math.pi * 4 * freq[targ_i] * t + 4 * pha_val[targ_i]),
                               np.cos(2 * math.pi * 4 * freq[targ_i] * t + 4 * pha_val[targ_i]),
                               np.sin(2 * math.pi * 5 * freq[targ_i] * t + 5 * pha_val[targ_i]),
                               np.cos(2 * math.pi * 5 * freq[targ_i] * t + 5 * pha_val[targ_i]),
                           ])
            # print(Y0.shape)
            Y[:, :, targ_i] = Y0;
            Y0 = [];
        #print(Y.shape)  # 10 50 40
        T1 = np.zeros([1040, 10, params['tw'], 40], np.float64)
        for i in range(0, 1040):
            T1[i, :, :, :] = Y;
        print(111)
        print(T1.shape)
        # T1 = T1.reshape((T1.shape[0], T1.shape[2], T1.shape[3], 10))
        T1 = T1.transpose(0, 2, 3, 1)
        #x_train = np.concatenate((x_train1, x_train2, x_train3), axis=-1)
    
        #    y_train = np.reshape(y_train, 3)
    
        ## build model
        # signal-CNN
        signal1 = Input(shape=(params['tw'],params['ch'],1))
        signal2 = Input(shape=(params['tw'], params['ch'], 1))
        signal3 = Input(shape=(params['tw'], params['ch'], 1))
        temp1 = Input(shape=(params['tw'], params['cl'], params['ch']))
        temp2 = Input(shape=(params['tw'], params['cl'], params['ch']))
        temp3 = Input(shape=(params['tw'], params['cl'], params['ch']))
        Y = Input(shape=(params['tw'], params['cl'], 10))
        conv11 = Conv2D(16, (9, 9), padding='same')(signal1)
        conv12 = Conv2D(1, (1, 9), padding='same')(conv11)
        conv13 = Conv2D(1, (1, 9), padding='valid')(conv12)
        drop1 = Dropout(0.5)(conv13)
        flat1 = Flatten()(drop1)
        flat11 = Flatten()(conv13)
        # reference-CNN
        conv21 = Conv2D(40, (9, 1), padding='same')(temp1)
        conv22 = Conv2D(1, (9, 1), padding='same')(conv21)
        drop2 = Dropout(0)(conv22)
        ##
        #conv21 = Conv2D(40, (1, 1), padding='same')(Y)
        # print("conv21.shape:" + str(conv21.shape))  # conv21.shape:(?, 50, 40, 40)
        conv31 = Conv2D(1, (1, 1), padding='same')(Y)
        #print(conv31)
        #print(flat11)
        m = Coff(params)([flat11, conv31])
        corr1 = Coff(params)([flat1, drop2])
        corr1 = Coff2(params)([m, corr1])
        ####################################
        conv11 = Conv2D(16, (9, 9), padding='same' )(signal2)
        conv12 = Conv2D(1, (1, 9), padding='same' )(conv11)
        conv13 = Conv2D(1, (1, 9), padding='valid' )(conv12)
        drop1 = Dropout(0.5)(conv13)
        flat1 = Flatten()(drop1)
        flat11 = Flatten()(conv13)
        # reference-CNN
        conv21 = Conv2D(40, (9, 1), padding='same' )(temp2)
        conv22 = Conv2D(1, (9, 1), padding='same' )(conv21)
        drop2 = Dropout(0 )(conv22)
    
        #conv21 = Conv2D(40, (1, 1), padding='same')(Y)
        # print("conv21.shape:" + str(conv21.shape))  # conv21.shape:(?, 50, 40, 40)
        conv31 = Conv2D(1, (1, 1), padding='same')(Y)
        m2 = Coff(params)([flat11, conv31])
        corr2 = Coff(params)([flat1, drop2])
        corr2 = Coff2(params)([m2, corr2])
        ##############################################################
        conv11 = Conv2D(16, (9, 9), padding='same')(signal3)
        conv12 = Conv2D(1, (1, 9), padding='same')(conv11)
        conv13 = Conv2D(1, (1, 9), padding='valid')(conv12)
        drop1 = Dropout(0.5)(conv13)
        flat1 = Flatten()(drop1)
        flat11 = Flatten()(conv13)
        # reference-CNN
        conv21 = Conv2D(40, (9, 1), padding='same')(temp3)
        conv22 = Conv2D(1, (9, 1), padding='same')(conv21)
        drop2 = Dropout(0 )(conv22)
        #conv21 = Conv2D(40, (1, 1), padding='same')(Y)
        # print("conv21.shape:" + str(conv21.shape))  # conv21.shape:(?, 50, 40, 40)
        conv31 = Conv2D(1, (1, 1), padding='same')(Y)
        m3 = Coff(params)([flat11, conv31])
        corr3 = Coff(params)([flat1, drop2])
        corr3 = Coff2(params)([m3, corr3])
        out=keras.layers.concatenate([corr1,corr2,corr3 ])
    
        out = Dense(params['cl'], activation='softmax')(out)
        #print("out.shape:" + str(out.shape))  # out.shape:(?, 40)
        model = Model(inputs=[signal1,signal2,signal3, temp1,temp2,temp3,Y],outputs=out)
        # 有一次提升, 则覆盖一次.
        model_path = './model/S'+str(subj)+'_'+str(params['tw'])+'.h5'
    
        #model_path = './fb_sign250S31_250.h5'
        model_checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        opt = keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, clipvalue=5.)
        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
    
        #print(model.summary())
    
        # train & test
        #x_train = x_train.astype('float32')
        #x_test = x_test.astype('float32')
        #print(y_train.shape)
        #print(np.tile(template2, (len(train_run),1,1,1)).shape)
        h=model.fit([x_train1 ,x_train2,x_train3 , np.tile(template, (len(train_run),1,1,1)), np.tile(template2, (len(train_run),1,1,1)),np.tile(template3, (len(train_run),1,1,1)),np.tile(T1, (len(train_run),1,1,1)) ], y_train, batch_size=32, epochs=20,
            validation_data=([x_test1 ,x_test2,x_test3 , template ,template2,template3,T1], y_test), shuffle=True,callbacks=[model_checkpoint],verbose=0)
        # h=model.fit([x_train1 ,x_train2,x_train3 , np.tile(template, (len(train_run),1,1,1)), np.tile(template2, (len(train_run),1,1,1)),np.tile(template3, (len(train_run),1,1,1)),np.tile(T1, (len(train_run),1,1,1)) ], y_train, batch_size=32, epochs=0,
        #     validation_data=([x_test1 ,x_test2,x_test3 , template ,template2,template3,T1], y_test), shuffle=True,verbose=0)
        #print(111)
        #print(111)
        model.load_weights(model_path)
    
        print("load successed")
        # Score trained model.
        scores = model.evaluate([x_test1 ,x_test2,x_test3 , template ,template2,template3,T1], y_test, verbose=1)
        y_pred = model.predict([x_test1 ,x_test2,x_test3 , template ,template2,template3,T1], batch_size=32)
        evaluation = {
                      'subject': subj,
                      'loss': scores[0],
                      # 'classifier_loss': classifier_loss,
                      'accuracy': scores[1],
                       }
    
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])
        csv_file = './fb_sign50.csv'

subject id is: 1
epoch第1次
111
(1040, 10, 250, 40)


2024-05-18 07:13:48.020717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)

Epoch 1: val_accuracy improved from -inf to 0.03365, saving model to ./model/S1_250.h5


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_accuracy improved from 0.03365 to 0.20288, saving model to ./model/S1_250.h5

Epoch 3: val_accuracy improved from 0.20288 to 0.54327, saving model to ./model/S1_250.h5

Epoch 4: val_accuracy improved from 0.54327 to 0.58077, saving model to ./model/S1_250.h5

Epoch 5: val_accuracy improved from 0.58077 to 0.68462, saving model to ./model/S1_250.h5

Epoch 6: val_accuracy improved from 0.68462 to 0.73462, saving model to ./model/S1_250.h5

Epoch 7: val_accuracy improved from 0.73462 to 0.77019, saving model to ./model/S1_250.h5

Epoch 8: val_accuracy improved from 0.77019 to 0.78846, saving model to ./model/S1_250.h5

Epoch 9: val_accuracy improved from 0.78846 to 0.82115, saving model to ./model/S1_250.h5

Epoch 10: val_accuracy improved from 0.82115 to 0.82692, saving model to ./model/S1_250.h5

Epoch 11: val_accuracy improved from 0.82692 to 0.84615, saving model to ./model/S1_250.h5

Epoch 12: val_accuracy improved from 0.84615 to 0.85192, saving model to ./model/S1_250

111
(1040, 10, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)
(None, 250, 40)


In [ ]:
from scipy.signal import cheb1ord, filtfilt, cheby1
file = loadmat('Bench/'+'Sub_1.mat')['data']
[N_channel,N_point, N_target, N_block] = np.shape(file)
fs=250;
file1=file
Wp = [6/fs*2, 90/fs*2]
Ws = [4/fs*2, 95/fs*2]
N, Wn=cheb1ord(Wp, Ws, 6, 18)
b, a = cheby1(N, 0.5, Wn,'bandpass')
for bloc_i in range(0,N_block):
        for targ_i in range(0,N_target):
            for chan_i in range(0,N_channel):
                file1[chan_i,:,targ_i,bloc_i] = filtfilt(b,a,file[chan_i,:,targ_i,bloc_i])



In [58]:
Coff(params)([flat11, conv31])


(None, 250, 40)


<KerasTensor shape=(None, 40), dtype=float32, sparse=False, name=keras_tensor_136>

In [51]:
[flat11, conv31]

[<KerasTensor shape=(None, 250), dtype=float32, sparse=False, name=keras_tensor_131>,
 <KerasTensor shape=(None, 250, 40, 1), dtype=float32, sparse=False, name=keras_tensor_135>]

In [65]:
import keras
print(keras.__version__)

3.3.3


In [67]:
tf.__version__

'2.16.1'

In [24]:
np.shape(x_train2)

(30000, 50, 9, 1)

In [25]:
np.shape(y_train)

(30000, 40)

In [11]:
import sys
print(sys.executable)


/usr/local/bin/python


ValueError: n_splits=4 cannot be greater than the number of members in each class.